In [ ]:
!pip install tldextract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 2.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 100)
import numpy as np
import re
import gdown
import os
import random
import tqdm
import pickle
import tldextract

In [ ]:
os.walk("/content/drive/MyDrive/nairaland_webtext")

<generator object _walk at 0x7c5a80f38660>

In [ ]:
files=[i for i in os.listdir("/content/drive/MyDrive/nairaland_webtext") if i.endswith(".csv")]

In [ ]:
len(files)

45

In [ ]:
len(os.listdir("/content/drive/MyDrive/nairaland_webtext") )

57

In [ ]:
def has_link(text):
    text = str(text)  # Ensure the input is a string
    url_pattern = re.compile(
        r'(?i)\bhttps?://\S+|www\.\S+|\b(?:ftp|http|https|mailto):\S+',
        re.IGNORECASE  # Case-insensitive matching
    )
    link_match = url_pattern.search(text)
    return bool(link_match)

import re

def has_link(text):
    text = str(text)
    # Enhanced regex pattern to match URLs
    url_pattern = r'https?://(?:www\.)?(?:[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b(?:[-a-zA-Z0-9@:%_\+.~#?&//=]*))'

    # Search for URLs in the text
    if re.search(url_pattern, text):
        return True
    else:
        return False

In [ ]:
def edit_section(filename):
    section=filename.split("_")[1]
    if section.endswith(".csv"):
        return section[:-4]
    return section

In [ ]:
edit_section("data_Food_2.csv")

'Food'

In [ ]:

def extract_links(text):
    # Regular expression pattern for URLs
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    url_pattern = re.compile(r'https?://(?:www\.)?(?:[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b(?:[-a-zA-Z0-9@:%_\+.~#?&//=]*))')

    # Find all matches in the text
    links = url_pattern.findall(text)
    all_links.extend(links)

In [ ]:
final_df=pd.DataFrame()
t_rows=0
for i,file in enumerate(files):
    print(i, file,t_rows)
    df_new=pd.read_csv(f'/content/drive/MyDrive/nairaland_webtext/{file}',lineterminator='\n')
    t_rows+=df_new.shape[0]


    #files that have link
    df_new=df_new[df_new["text"].apply(has_link)]

    #extract the link
    all_links=[]
    df_new["text"].apply(extract_links)

    #create link df
    links_df=pd.DataFrame({"link":all_links})
    links_df.drop_duplicates(inplace=True)
    links_df["section"]=edit_section(file)

    final_df=pd.concat([final_df,links_df])#, inplace=True)

0 data_Food.csv 0
1 data_Programming.csv 246748
2 data_Career.csv 451938
3 data_Business.csv 832312
4 data_Phones.csv 1074626
5 data_Education.csv 1530440
6 data_Romance.csv 2412898
7 data_Religion.csv 4248009
8 data_Phones_3.csv 4759401
9 data_Religion_2.csv 5145107
10 data_Education_2.csv 5778251
11 data_Romance_2.csv 6156144
12 data_Phones_2.csv 7245797
13 data_Business_2.csv 7782744
14 data_Phones_4.csv 8147355
15 data_Literature.csv 8216045
16 data_Education_3.csv 8263006
17 data_Fashion.csv 8359054
18 data_Romance_3.csv 8684784
19 data_Religion_3.csv 9824430
20 data_Religion_4.csv 10417162
21 data_Politics.csv 10497795
22 data_Politics_2.csv 12538273
23 data_Politics_3.csv 13500104
24 data_TV-Movies.csv 13902502
25 data_Travel.csv 14355290
26 data_Culture.csv 15110161
27 data_Romance_4.csv 15400246
28 data_Music-Radio.csv 16357724
29 data_Family.csv 16725924
30 data_Sports_4.csv 17936291
31 data_Business_3.csv 18162563
32 data_Politics_4.csv 18933444
33 data_Car Talk.csv 20102754

In [ ]:
final_df.shape

(1959094, 2)

In [ ]:
t_rows

28773427

In [ ]:
#df=pd.read_csv('/content/drive/MyDrive/nairaland_webtext/data_Politics_7.csv',lineterminator='\n')

In [ ]:
final_df["section"].value_counts()

,count
section,
Politics,814088
Sports,146571
Education,124488
Religion,119967
Romance,119266
Business,99728
Phones,90820
Music-Radio,83501
Health,82054


In [ ]:
final_df.drop_duplicates("link", inplace=True)

In [ ]:
final_df["section"].value_counts()

,count
section,
Politics,726968
Sports,143434
Education,120464
Romance,114537
Religion,112761
Business,93002
Music-Radio,81106
Health,76517
Phones,61737


In [ ]:
final_df.shape

(1795908, 2)

In [ ]:
  # domains that aren't scraper friendly. do not include subdomains!
exclude_domains = set([
    # image & video hosting sites
    'imgur.com',
    'redd.it',
    'instagram.com',
    'discord.gg',
    'gfycat.com',
    'giphy.com',
    'reddituploads.com',
    'redditmedia.com',
    'twimg.com',
    'sli.mg',
    'magaimg.net',
    'flickr.com',
    'imgflip.com',
    'youtube.com',
    'youtu.be',
    'youtubedoubler.com',
    'vimeo.com',
    'twitch.tv',
    'streamable.com',
    'bandcamp.com',
    'soundcloud.com',

    # not scraper friendly

    'reddit.com',
    'gyazo.com',
    'github.com',
    'xkcd.com',
    'twitter.com',
    'x.com',
    'spotify.com',
    'itunes.apple.com',
    'facebook.com',
    "fb.watch",
    'gunprime.com',
    'strawpoll.me',
    'voyagefusion.com',
    'rollingstone.com',
    'google.com',
    'timeanddate.com',
    'walmart.com',
    'roanoke.com',
    'spotrac.com',

    # original paper excluded wikipedia
    'wikipedia.org',
  'nairaland.com',

    # lots of top posts for this one
    'battleforthenet.com',
])

In [ ]:
exclude_extensions = (
    '.png',
    '.jpg',
    '.jpeg',
    '.gif',
    '.gifv',
    '.pdf',
    '.mp4',
    '.mp3',
    '.ogv',
    '.webm',
    '.doc',
    '.docx',
    '.log',
    '.csv',
    '.dat',
    '.iso',
    '.bin',
    '.exe',
    '.apk',
    '.jar',
    '.app',
    '.ppt',
    '.pps',
    '.pptx',
    '.xml',
    '.gz',
    '.xz',
    '.bz2',
    '.tgz',
    '.tar',
    '.zip',
    '.wma',
    '.mov',
    '.wmv',
    '.3gp',
    '.svg',
    '.rar',
    '.wav',
    '.avi',
    '.7z'
)

In [ ]:
def should_exclude(url):
    ext = tldextract.extract(url)
    #print(ext)
    domain = ext.domain+'.'+ext.suffix


    #basedomain = '.'.join(ext[-2:])


    # Ignore excluded domains
    if domain in exclude_domains:
        return True

    # Ignore case-insensitive matches for excluded extensions
    if url.lower().split('?')[0].endswith(exclude_extensions):
        return True

    return False

In [ ]:
final_df=final_df[~final_df["link"].apply(should_exclude)]

In [ ]:
final_df.shape

#### Trailing full stop

In [ ]:
def remove_trailing_fullstop(link):
  while link[-1]==".":
    link=link[:-1]
  return link

def has_trailing_fullstop(link):
  if link[-1]==".":
    return True
  return False

In [ ]:
final_df[final_df["link"].apply(has_trailing_fullstop)]

In [ ]:
final_df["link"]=final_df["link"].apply(remove_trailing_fullstop)

In [ ]:
final_df[final_df["link"].apply(has_trailing_fullstop)]

,link,section


In [ ]:
final_df.drop_duplicates("link", inplace=True)

In [ ]:
final_df.shape

(1289195, 2)

#### consecutive full stop

In [ ]:
def remove_consecutive_fullstop(link):
  if ".." in link:
    #print(1)
    return link.replace("..",".")
  return link

def has_consecutive_fullstop(link):
  if ".." in link:
    return True
  return False

In [ ]:
final_df[final_df["link"].apply(has_consecutive_fullstop)]

,link,section
334,https://dreamchasm..com/2024/07/how-to-make-ca...,Food
409,http://www..com.ng/index.php/more-news/life-st...,Food
644,http://auntysazzy..de/2013/10/home-made-bread-...,Food
771,http://opeyemioladotun..com/2015/08/for-love-o...,Food
779,https://foodprepeasy..com/2024/04/diy-to-prepa...,Food
...,...,...
95012,http://punctualnewsblog..com/2020/11/i-will-sa...,Politics
95014,http://lindaikeji..com/2015/04/graphic-photo-p...,Politics
95097,https://all-gists..com/2018/12/2019-presidenti...,Politics
95111,https://whenhow24..com/2019/10/chei-abeg-comot...,Politics


In [ ]:
100*84442/1291168

6.539970011648368

In [ ]:
final_df["link"]=final_df["link"].apply(remove_consecutive_fullstop)

In [ ]:
final_df.drop_duplicates("link", inplace=True)

In [ ]:
final_df.head(5)

,link,section
38,http://reshade.com/resize/constrain/image-5154...,Food
39,http://reshade.com/resize/constrain/image-5154...,Food
40,http://reshade.com/resize/constrain/image-5154...,Food
41,http://reshade.com/resize/constrain/image-5175...,Food
42,http://reshade.com/resize/constrain/image-5175...,Food


In [ ]:
final_df=final_df.sample(frac=1,random_state=0)

In [ ]:
final_df.shape

In [ ]:
final_df.head(5)

,link,section
40539,http://www.baricblog.com/2013/01/start-trading...,Career
30495,http://saharareporters.com/2021/05/20/break-ni...,Politics
44048,http://accountingsoftware.com.ng/mortgage-back...,Business
34095,http://www.punchng.com/news/pdp-primaries-jona...,Politics
64944,https://www.vanguardngr.com/2020/12/i-saw-over...,Politics


In [ ]:
final_df.to_csv("/content/drive/MyDrive/nairaland_webtext/links/final_links.csv",index=False)